# Explorando as estatísticas de gênero do Banco Mundial

Seguiremos investigando as base do Banco Mundial de estatísticas relacionadas a gênero. 

Hoje vamos explorar a série histórica das taxas de fecundidade entre jovens, agrupadas por região geográfica, que foram utilizadas no artigo [The Social and Educational Consequences of Adolescent Childbearing](https://genderdata.worldbank.org/data-stories/adolescent-fertility/).


In [18]:
import wbgapi as wb
import pandas as pd

Vamos buscar e organizar os códigos e nomes das regiões geográficas da base de dados.

In [19]:
region_info = wb.region.info()

A partir do objeto retornado pela consulta, crie um dicionário chamado **region_dict**. Este dicionário conterá **dois** itens. Suas chaves são **region_code** e **region_name**. Seus valores são listas. A lista da chave **region_code** contém os códigos das regiões geográficas, e a lista da chave **region_name** contém os nomes das regiões geográficas. 

*dica*: explore o atributo **items** do objeto **region_info**.

In [20]:
region_dict = {"region_code": [region.get('code') for region in region_info.items],
               "region_name": [region.get('name') for region in region_info.items]}

In [21]:
region_dict

{'region_code': ['AFE',
  'AFR',
  'AFW',
  'ARB',
  'CAA',
  'CEA',
  'CEB',
  'CEU',
  'CLA',
  'CME',
  'CSA',
  'CSS',
  'EAP',
  'EAR',
  'EAS',
  'ECA',
  'ECS',
  'EMU',
  'EUU',
  'FCS',
  'HPC',
  'LAC',
  'LCN',
  'LDC',
  'LTE',
  'MDE',
  'MEA',
  'MNA',
  'NAC',
  'NAF',
  'NRS',
  'OED',
  'OSS',
  'PRE',
  'PSS',
  'PST',
  'RRS',
  'SAS',
  'SSA',
  'SSF',
  'SST',
  'SXZ',
  'WLD',
  'XZN'],
 'region_name': ['Africa Eastern and Southern',
  'Africa',
  'Africa Western and Central',
  'Arab World',
  'Sub-Saharan Africa (IFC classification)',
  'East Asia and the Pacific (IFC classification)',
  'Central Europe and the Baltics',
  'Europe and Central Asia (IFC classification)',
  'Latin America and the Caribbean (IFC classification)',
  'Middle East and North Africa (IFC classification)',
  'South Asia (IFC classification)',
  'Caribbean small states',
  'East Asia & Pacific (excluding high income)',
  'Early-demographic dividend',
  'East Asia & Pacific',
  'Europe & C

Com este formato de dicionário, você consegue facilmente criar um DataFrame Pandas. A função pandas.Dataframe() automaticamente interpreta as chaves do dicionário como os nomes das colunas, e os valores como as linhas. 

Crie o DataFrame **region_df** a partir do dicionário **region_dict**.

In [22]:
region_df = pd.DataFrame(region_dict)

Vamos consultar a série histórica completa 'SP.ADO.TFRT' referente à taxa de fecundidade entre jovens de 15 a 19 anos. 

Para restringir a consulta a somente regiões geográficas (não queremos que a consulta retorne dados de países ou de grupos de renda), vamos passar, no argumento **economy** da função, os códigos das regiões. 

Você pode acessá-los diretamente do DataFrame ou do dicionário. 

In [23]:
serie_afr = wb.data.DataFrame(series = ['SP.ADO.TFRT'], 
                              # economy =  ,
                              economy = region_df['region_code'],
                              time = "all")

Investigue a base de dados. O formato padrão de retorno dessa consulta é um DataFrame com os códigos de país/região nos índices e os anos nas colunas. 

Exclua do DataFrame as colunas em que todos os valores são indisponíveis (no caso, **NaN**).

In [24]:
serie_afr.dropna(axis=1, how = 'all', inplace = True)

Vamos reorganizar este DataFrame de modo a manter as regiões como índices, os anos em uma coluna **year** e as taxas em uma coluna **af_rate**.

In [40]:
serie_afr_long = serie_afr.melt(var_name = "year", 
                                value_name = "af_rate", 
                                ignore_index = False)

Agora entra em cena o DataFrame **region_df** que você criou há pouco. Adicione ao DataFrame de taxas de fecundidade uma coluna com os respectivos nomes das regiões. 

Após a junção das tabelas, retorne a coluna com os códigos das regiões ao índice do DataFrame.

In [41]:
serie_afr_long = (serie_afr_long
                 .merge(right = region_df, 
                        how = "inner", 
                        left_on = "economy", 
                        right_on = "region_code")
                 .set_index("region_code"))

Por fim, formate os valores da coluna **year**, mantendo somente os números.

In [38]:
# uma opção

serie_afr_long["year"] = serie_afr_long.year.map(lambda s: s.strip("YR"))

In [ ]:
# outra opção

serie_afr_long["year"] = serie_afr_long.year.str.strip("YR")

In [42]:
# outra opção

serie_afr_long = serie_afr_long.assign(year = lambda y: y['year'].str.strip("YR"))

# Explorando as estatísticas brasileiras de comércio exterior.

Vamos criar um DataFrame com as importações, exportações e saldo comercial bilateral do Brasil com seus parceiros comerciais, de 2012 a 2022, em milhões de dólares americanos.

Primeiro, crie os DataFrames **serie_imp** e **serie_exp** a partir dos arquivos .csv disponíveis na pasta **./data/**. Os arquivos contêm os valores *Free on Board* em dólares americanos das importações/exportações por país e ano.

In [2]:
serie_imp = pd.read_csv("data/IMP_2012_2022_20230921.csv", sep = ";")
serie_exp = pd.read_csv("data/EXP_2012_2022_20230921.csv", sep = ";")

In [5]:
serie_imp['vl_usd_milhoes'] = serie_imp['valor_fob_usd'] / 1e6

Crie as colunas **vl_usd_milhoes**, com os valores em milhões de dólares, em ambos DataFrames. Remove as colunas de valor originais.

In [6]:
serie_imp['vl_usd_milhoes'] = serie_imp['valor_fob_usd'].map(lambda x: x / 1e6)
serie_exp['vl_usd_milhoes'] = serie_exp['valor_fob_usd'].map(lambda x: x / 1e6)

serie_imp.drop('valor_fob_usd', axis = 1, inplace = True)
serie_exp.drop('valor_fob_usd', axis = 1, inplace = True)

In [12]:
serie_exp

,ano,país,vl_usd_milhoes
0,2022,China,89427.761284
1,2022,Estados Unidos,37437.814354
2,2022,Argentina,15344.651930
3,2022,Países Baixos (Holanda),11927.910064
4,2022,Espanha,9747.538076
...,...,...,...
2601,2012,"Wake, Ilha",0.003191
2602,2012,"Madeira, Ilha da",0.002392
2603,2012,"Canal, Ilhas do (Guernsey)",0.001305
2604,2012,"Marianas do Norte, Ilhas",0.001092


Junte os dois objetos no DataFrame **serie_exp_imp**, contendo as colunas **ano**, **país**, **vl_usd_milhoes_exp** e **vl_usd_milhoes_imp**.

Em seguida, crie a coluna **saldo_usd_milhoes**, com a diferença entre as exportações e as importações por país e ano. 

**Atenção**: alguns países podem estar presentes na série de exportações e não na série de importações, e vice-versa. Queremos que o DataFrame final contenha todos os parceiros comerciais, inclusive os para os quais o Brasil apenas exportou ou dos quais apenas importou.

*dica*: após a junção dos DataFrames, as combinações de chaves que estavam presente em um DataFrame e ausentes em outro podem gerar linhas com NaN.

In [14]:
serie_exp_imp = (serie_exp
                 .merge(serie_imp, 
                        on = ["ano", "país"], 
                        suffixes = ["_exp", "_imp"], 
                        how = "outer")
                 .fillna(value = int(0))
                 .assign(saldo_usd_milhoes = lambda x: x['vl_usd_milhoes_exp'] - x['vl_usd_milhoes_imp']))

Agora, confira quais foram os parceiros comerciais com os quais o Brasil teve o maior saldo no ano de 2022. 

Crie um novo DataFrame, **maiores_saldos_2022**, mantendo todas as suas colunas do anterior e apenas os 10 maiores saldos naquele ano.

In [ ]:
maiores_saldos_2020 = (serie_exp_imp
                       .query('ano == 2022')
                       .sort_values(by = "saldo_usd_milhoes", ascending = False)
                       .head(10))